In [ ]:
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import pandas as pd
import numpy as np
import pickle
import math

In [ ]:
seed = 99

# Load Model

In [ ]:
### Import Data ###
dataset = pd.read_excel("/home/admin2/ML_NIC/datasets/land+mines-1/Mine_Dataset.xls", 
                        sheet_name="Normalized_Data", names=['Voltage', 'Height', 'Soil Type', 'Mine Type'])

In [ ]:
dataset_features = dataset.drop(columns=['Mine Type'])
dataset_labels = dataset['Mine Type']

In [ ]:
### Build Train and Test Sets ###
X_train, X_test, y_train, y_test = train_test_split(dataset_features, dataset_labels, 
                                                    random_state=seed, test_size=0.2,
                                                    stratify=dataset_labels)

In [ ]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
one_hot = OneHotEncoder(handle_unknown='ignore')
X_train = np.hstack((np.delete(X_train, 2, 1), one_hot.fit_transform(X_train[:, 2].reshape(-1, 1)).toarray()))
X_test = np.hstack((np.delete(X_test, 2, 1), one_hot.transform(X_test[:, 2].reshape(-1, 1)).toarray()))

In [ ]:
# Load up model
with open("model.pkl", 'rb') as f:
    clf = pickle.load(f)

# Micro-C Model

In [ ]:
n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold
values = clf.tree_.value

node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
while len(stack) > 0:
    # `pop` ensures each node is only visited once
    node_id, depth = stack.pop()
    node_depth[node_id] = depth

    # If the left and right child of a node is not the same we have a split
    # node
    is_split_node = children_left[node_id] != children_right[node_id]
    # If a split node, append left and right children and depth to `stack`
    # so we can loop through them
    if is_split_node:
        stack.append((children_left[node_id], depth + 1))
        stack.append((children_right[node_id], depth + 1))
    else:
        is_leaves[node_id] = True

print(
    "The binary tree structure has {n} nodes and has "
    "the following tree structure:\n".format(n=n_nodes)
)
for i in range(n_nodes):
    if is_leaves[i]:
        print(
            "{space}node={node} is a leaf node with value={value}.".format(
                space=node_depth[i] * "\t", node=i, value=values[i]
            )
        )
    else:
        print(
            "{space}node={node} is a split node with value={value}: "
            "go to node {left} if X[:, {feature}] <= {threshold} "
            "else to node {right}.".format(
                space=node_depth[i] * "\t",
                node=i,
                left=children_left[i],
                feature=feature[i],
                threshold=threshold[i],
                right=children_right[i],
                value=values[i],
            )
        )

In [ ]:
# Decide number of bits required for the path storage
num_features = clf.n_features_in_
classes = clf.classes_
num_leaves = int(np.sum(is_leaves))
num_result_words = math.ceil(num_leaves/32.0)
result_init = hex((2**(num_leaves % 32))-1)
features_per_result_core = int(32.0/num_result_words)
num_result_cores = [math.ceil(num_features / features_per_result_core)]

while(num_result_cores[-1] != 1):
    num_result_cores.append(math.ceil(num_result_cores[-1] / features_per_result_core))

    

In [ ]:
total_bits = 16
dec_bits = 13

def num_translate(number, total_bits, dec_bits): 
    return round(number * (2**dec_bits))

In [ ]:
print("__lmem uint32_t path_class[] = {", end='')
for i in range(is_leaves.shape[0]):
    if (is_leaves[i]):
        print(f"{classes[np.argmax(values[i][0])]}", end=', ')
print('};')

In [ ]:
# Depth-first search through nodes
def depth_first(root, left, right, prediction, feature, threshold, output, feature_id, num_result_words, feat_file):
    if (left[root] == right[root]):  # leaf node is base case
        if (output == ''):
            return
        
        if feat_file:
            path_id = np.sum(is_leaves[:root])
            output = output.lstrip("&& ")
            output = output.rstrip(" ")

            if (num_result_words > 1):
                result_index = num_result_words - int(path_id/32) - 1
                path_id = path_id % 32
                feat_file.write(f"        if (!({output}))\n            result_gpr[{result_index}] &= ~(1 << {path_id});\n")
            else:
                feat_file.write(f"        if (!({output}))\n            result_gpr &= ~(1 << {path_id});\n")
            feat_file.write("\n")
            
        else:
            path_id = np.sum(is_leaves[:root])
            output = output.lstrip("and ")
            output = output.rstrip(" ")

            if (num_result_words > 1):
                result_index = num_result_words - int(path_id/32) - 1
                path_id = path_id % 32
                print(f"    if (not({output})):\n        result_gpr[{result_index}] &= ~(1 << {path_id})")
            else:
                print(f"    if (not({output})):\n        result_gpr &= ~(1 << {path_id})")
            print("")
            
    else:
        
        if feat_file:
            net_threshold = num_translate(threshold[root], total_bits, dec_bits)
            if (feature[root] == feature_id):
                left_output = output + f"&& feature <= {net_threshold} "
                right_output = output + f"&& feature > {net_threshold} "
            else:
                left_output = output
                right_output = output

            depth_first(left[root], left, right, prediction, feature, threshold, 
                        left_output, feature_id, num_result_words, feat_file)
            depth_first(right[root], left, right, prediction, feature, threshold, 
                        right_output, feature_id, num_result_words, feat_file)
            
        else:
            net_threshold = num_translate(threshold[root], total_bits, dec_bits)
            if (feature[root] == feature_id):
                left_output = output + f"and feature <= {net_threshold} "
                right_output = output + f"and feature > {net_threshold} "
            else:
                left_output = output
                right_output = output

            depth_first(left[root], left, right, prediction, feature, threshold, 
                        left_output, feature_id, num_result_words, feat_file)
            depth_first(right[root], left, right, prediction, feature, threshold, 
                        right_output, feature_id, num_result_words, feat_file)

        

In [ ]:
for feat_id in range(num_features):
    print(f"def feature{feat_id+1}(feature):")
    
    if (num_result_words > 1):
        print(f"    result_gpr = [{result_init}", end='')
        for i in range(1, num_result_words):
            print(", 0xffffffff", end='')
        print(']')
    else:
        print(f"    result_gpr = {result_init}")
        
    depth_first(0, children_left, children_right, values, feature, threshold, "", feat_id, num_result_words, None)
    print("    return result_gpr")
    print()
    print()

# Test Micro-C Model (Python Version)

In [ ]:
def feature1(feature):
    result_gpr = [0x3ffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 4789 and feature <= 2810 and feature <= 2245)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature <= 4789 and feature <= 2810 and feature <= 2245)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature <= 4789 and feature <= 2810 and feature <= 2245)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2315)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2315)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature > 2315)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2446)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature > 2446)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature > 2446)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2388 and feature <= 2314)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2388 and feature > 2314)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature > 2388)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2771 and feature <= 2759)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2771 and feature > 2759)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2771)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature > 2771)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature <= 2357)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature <= 2357 and feature <= 2314)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature <= 2357 and feature > 2314)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature <= 2611 and feature <= 2450)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature <= 2611 and feature <= 2450 and feature <= 2422)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature <= 2611 and feature <= 2450 and feature > 2422)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature <= 2611 and feature > 2450 and feature <= 2518 and feature <= 2481)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature <= 2611 and feature > 2450 and feature <= 2518 and feature > 2481)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature <= 2611 and feature > 2450 and feature > 2518)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature <= 2611 and feature > 2450 and feature > 2518 and feature <= 2537)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature <= 2611 and feature > 2450 and feature > 2518 and feature > 2537)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature <= 2611 and feature > 2450 and feature > 2518)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature <= 2611 and feature > 2450)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature > 2611 and feature <= 2636)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature > 2611 and feature > 2636)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature > 2611)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature <= 2518)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature <= 2661 and feature > 2357 and feature > 2518)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature > 2661 and feature <= 2740 and feature <= 2685)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature > 2661 and feature <= 2740 and feature > 2685)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature > 2661 and feature <= 2740 and feature > 2685 and feature <= 2710)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature > 2661 and feature <= 2740 and feature > 2685 and feature > 2710)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature > 2661 and feature > 2740)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature > 2661 and feature > 2740)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature > 2661 and feature > 2740)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature <= 4789 and feature <= 2810 and feature > 2245 and feature > 2661)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature <= 3033)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 3033)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 3033)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature <= 3057)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature <= 3057)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 3057 and feature <= 3479 and feature <= 3172 and feature <= 3135)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 3057 and feature <= 3479 and feature <= 3172 and feature > 3135)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 3057 and feature <= 3479 and feature > 3172)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 3057 and feature > 3479)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 3057)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature <= 2859)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 2859 and feature <= 3304 and feature <= 2874)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 2859 and feature <= 3304 and feature > 2874 and feature <= 2897)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 2859 and feature <= 3304 and feature > 2874 and feature > 2897 and feature <= 3267 and feature <= 3007)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 2859 and feature <= 3304 and feature > 2874 and feature > 2897 and feature <= 3267 and feature <= 3007)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 2859 and feature <= 3304 and feature > 2874 and feature > 2897 and feature <= 3267 and feature > 3007)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 2859 and feature <= 3304 and feature > 2874 and feature > 2897 and feature > 3267)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 2859 and feature <= 3304 and feature > 2874)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 2859 and feature <= 3304 and feature > 2874)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3626 and feature > 2859 and feature > 3304)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 4789 and feature > 2810 and feature > 3626 and feature <= 4591 and feature <= 4343)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 4789 and feature > 2810 and feature > 3626 and feature <= 4591 and feature > 4343)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 4789 and feature > 2810 and feature > 3626 and feature <= 4591)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature <= 4789 and feature > 2810 and feature > 3626 and feature <= 4591 and feature <= 3836)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature <= 4789 and feature > 2810 and feature > 3626 and feature <= 4591 and feature > 3836 and feature <= 4207)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 4789 and feature > 2810 and feature > 3626 and feature <= 4591 and feature > 3836 and feature > 4207)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 4789 and feature > 2810 and feature > 3626 and feature > 4591)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 4789 and feature > 2810 and feature > 3626)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature <= 4789 and feature > 2810 and feature > 3626 and feature <= 3750)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature <= 4789 and feature > 2810 and feature > 3626 and feature > 3750)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature <= 4789 and feature > 2810 and feature > 3626)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature <= 4789 and feature > 2810 and feature > 3626)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3134)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature <= 4789 and feature > 2810 and feature > 3134 and feature <= 3783)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature <= 4789 and feature > 2810 and feature > 3134 and feature > 3783)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature <= 4789 and feature > 2810 and feature > 3134 and feature <= 4009)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature <= 4789 and feature > 2810 and feature > 3134 and feature > 4009)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature <= 4789 and feature > 2810 and feature <= 2945 and feature <= 2871)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature <= 4789 and feature > 2810 and feature <= 2945 and feature > 2871)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature <= 3923)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature > 3923 and feature <= 4294)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature > 3923 and feature > 4294)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature <= 3527 and feature <= 3329)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature <= 3527 and feature > 3329)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature <= 3527)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature <= 3527)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature <= 3527 and feature <= 3007)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature <= 3527 and feature > 3007)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature > 3527 and feature <= 3762)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature > 3527 and feature > 3762 and feature <= 4034)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature > 3527 and feature > 3762 and feature > 4034 and feature <= 4170)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature > 3527 and feature > 3762 and feature > 4034 and feature > 4170 and feature <= 4282)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature > 3527 and feature > 3762 and feature > 4034 and feature > 4170 and feature > 4282)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature > 3527 and feature > 3762 and feature > 4034 and feature > 4170)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature > 3527 and feature > 3762 and feature > 4034)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature > 3527 and feature > 3762)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature <= 4789 and feature > 2810 and feature > 2945 and feature > 3527)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3417 and feature <= 2931)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3417 and feature > 2931)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3417 and feature > 2931)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3417 and feature > 2931)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3417 and feature > 2931)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature <= 4789 and feature > 2810 and feature <= 3417 and feature > 2931)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature <= 4789 and feature > 2810 and feature > 3417)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature > 4789 and feature <= 5061)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature > 4789 and feature <= 5061)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature > 4789 and feature <= 5061)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature > 4789 and feature > 5061)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature > 4789 and feature > 5061)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature > 4789 and feature > 5061)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 4789 and feature > 5061)):
        result_gpr[0] &= ~(1 << 17)

    return result_gpr


def feature2(feature):
    result_gpr = [0x3ffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 5585)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature <= 5585)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature <= 5585)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature <= 5585)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature <= 5585 and feature <= 4841)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature <= 5585 and feature > 4841)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature <= 5585 and feature <= 3351 and feature <= 1117)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature <= 5585 and feature <= 3351 and feature > 1117)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature <= 5585 and feature <= 3351 and feature > 1117)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature <= 5585 and feature <= 3351 and feature > 1117)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature <= 5585 and feature > 3351 and feature <= 4096)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature <= 5585 and feature > 3351 and feature <= 4096)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature <= 5585 and feature > 3351 and feature > 4096)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature <= 5585 and feature > 3351)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature > 5585)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature > 5585)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature > 5585)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature > 5585)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature > 5585)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature > 5585)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature > 5585)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature > 5585)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature > 5585 and feature <= 7075)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature > 5585 and feature > 7075 and feature <= 7820)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature > 5585 and feature > 7075 and feature <= 7820)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature > 5585 and feature > 7075 and feature > 7820)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 5585)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature > 5585 and feature <= 7075)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature > 5585 and feature <= 7075)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature > 5585 and feature > 7075)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature > 5585)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature > 5585)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature > 5585)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature > 5585)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature > 5585)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature > 5585)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature > 5585 and feature <= 7075)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature > 5585 and feature > 7075)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature > 5585 and feature > 7075)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature > 5585)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature <= 6330 and feature <= 1117)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature <= 6330 and feature <= 1117)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature <= 6330 and feature <= 1117)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature <= 6330 and feature > 1117 and feature > 4096)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature <= 6330 and feature > 1117 and feature > 4096)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature <= 6330 and feature > 1117 and feature > 4096 and feature <= 5585)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 6330 and feature > 1117 and feature > 4096 and feature <= 5585 and feature <= 4841)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 6330 and feature > 1117 and feature > 4096 and feature <= 5585 and feature > 4841)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 6330 and feature > 1117 and feature > 4096 and feature <= 5585)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature <= 6330 and feature > 1117 and feature > 4096 and feature <= 5585)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature <= 6330 and feature > 1117 and feature > 4096 and feature <= 5585)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature <= 6330 and feature > 1117 and feature > 4096 and feature > 5585)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature <= 6330 and feature > 1117 and feature > 4096)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 6330 and feature <= 2607 and feature <= 372)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 6330 and feature <= 2607 and feature <= 372)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 6330 and feature <= 2607 and feature > 372 and feature <= 1862)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature <= 6330 and feature <= 2607 and feature > 372 and feature > 1862)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature <= 6330 and feature <= 2607 and feature > 372 and feature > 1862)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 6330 and feature <= 2607 and feature > 372 and feature > 1862)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 6330 and feature <= 2607)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 6330 and feature <= 2607 and feature <= 372)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature <= 6330 and feature <= 2607 and feature > 372)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature <= 6330 and feature <= 2607 and feature > 372)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature <= 6330 and feature > 2607)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature <= 6330 and feature > 2607)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature <= 6330)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature <= 6330 and feature <= 1862)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature <= 6330 and feature <= 1862)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature <= 6330 and feature > 1862)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature <= 6330 and feature > 1862)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature <= 6330)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature <= 6330)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature <= 6330 and feature <= 1117)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature <= 6330 and feature <= 1117)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature <= 6330 and feature <= 1117)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096 and feature <= 2234)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096 and feature <= 2234)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096 and feature > 2234)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature <= 6330 and feature > 1117 and feature > 4096 and feature <= 5585)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature <= 6330 and feature > 1117 and feature > 4096 and feature > 5585)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature <= 6330 and feature > 1117 and feature > 4096 and feature > 5585)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096 and feature <= 3351)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096 and feature <= 3351 and feature <= 2607)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096 and feature <= 3351 and feature <= 2607 and feature <= 1862)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096 and feature <= 3351 and feature <= 2607 and feature <= 1862)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096 and feature <= 3351 and feature <= 2607 and feature > 1862)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096 and feature <= 3351 and feature > 2607)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature <= 6330 and feature > 1117 and feature <= 4096 and feature > 3351)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature <= 6330 and feature > 1117 and feature > 4096)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature > 6330)):
        result_gpr[0] &= ~(1 << 4)

    if (not(feature > 6330 and feature <= 7447)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature > 6330 and feature <= 7447)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature > 6330 and feature > 7447)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature > 6330)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 6330)):
        result_gpr[0] &= ~(1 << 9)

    if (not(feature > 6330)):
        result_gpr[0] &= ~(1 << 10)

    if (not(feature <= 3724 and feature <= 745)):
        result_gpr[0] &= ~(1 << 11)

    if (not(feature <= 3724 and feature > 745)):
        result_gpr[0] &= ~(1 << 12)

    if (not(feature > 3724)):
        result_gpr[0] &= ~(1 << 13)

    if (not(feature <= 1117 and feature <= 372)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature <= 1117 and feature > 372)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 1117)):
        result_gpr[0] &= ~(1 << 17)

    return result_gpr


def feature3(feature):
    result_gpr = [0x3ffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 9)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 10)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 11)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 12)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 13)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 14)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 15)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 16)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature > 4096)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature > 4096)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature > 4096)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 8)

    return result_gpr


def feature4(feature):
    result_gpr = [0x3ffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 17)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 18)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 19)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature > 4096)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature > 4096)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 15)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 16)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 17)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 18)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 19)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 20)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 21)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 22)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 23)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 24)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 25)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 26)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 27)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 28)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 29)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 30)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 31)

    if (not(feature > 4096)):
        result_gpr[0] &= ~(1 << 0)

    if (not(feature > 4096)):
        result_gpr[0] &= ~(1 << 1)

    if (not(feature > 4096)):
        result_gpr[0] &= ~(1 << 2)

    if (not(feature > 4096)):
        result_gpr[0] &= ~(1 << 3)

    if (not(feature <= 4096)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature <= 4096)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 4096)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature > 4096)):
        result_gpr[0] &= ~(1 << 8)

    return result_gpr


def feature5(feature):
    result_gpr = [0x3ffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 3)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 4)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 5)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 6)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 7)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature > 4096)):
        result_gpr[2] &= ~(1 << 10)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature > 4096)):
        result_gpr[2] &= ~(1 << 20)

    return result_gpr


def feature6(feature):
    result_gpr = [0x3ffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 23)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 24)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 25)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 26)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 27)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 28)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 29)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 30)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 31)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 0)

    if (not(feature > 4096)):
        result_gpr[2] &= ~(1 << 1)

    if (not(feature > 4096)):
        result_gpr[2] &= ~(1 << 2)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature <= 4096)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature <= 4096)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 4096)):
        result_gpr[0] &= ~(1 << 7)

    if (not(feature <= 4096)):
        result_gpr[0] &= ~(1 << 8)

    if (not(feature > 4096)):
        result_gpr[0] &= ~(1 << 9)

    return result_gpr


def feature7(feature):
    result_gpr = [0x3ffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 0)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 1)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 2)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 20)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 21)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 22)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 11)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 12)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 13)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 14)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 15)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 16)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 17)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 18)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 19)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 20)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 21)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 22)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 23)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 24)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 25)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 26)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 27)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 28)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 29)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 30)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 31)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 0)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 1)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 2)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 3)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 4)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 5)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 6)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 7)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 8)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 9)

    if (not(feature <= 4096)):
        result_gpr[1] &= ~(1 << 10)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 11)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 12)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 13)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 14)

    if (not(feature > 4096)):
        result_gpr[1] &= ~(1 << 15)

    return result_gpr


def feature8(feature):
    result_gpr = [0x3ffff, 0xffffffff, 0xffffffff, 0xffffffff]
    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 3)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 4)

    if (not(feature <= 4096)):
        result_gpr[3] &= ~(1 << 5)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 6)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 7)

    if (not(feature > 4096)):
        result_gpr[3] &= ~(1 << 8)

    if (not(feature <= 4096)):
        result_gpr[2] &= ~(1 << 8)

    if (not(feature > 4096)):
        result_gpr[2] &= ~(1 << 9)

    if (not(feature <= 4096)):
        result_gpr[0] &= ~(1 << 5)

    if (not(feature > 4096)):
        result_gpr[0] &= ~(1 << 6)

    if (not(feature <= 4096)):
        result_gpr[0] &= ~(1 << 14)

    if (not(feature > 4096)):
        result_gpr[0] &= ~(1 << 15)

    if (not(feature > 4096)):
        result_gpr[0] &= ~(1 << 16)

    if (not(feature > 4096)):
        result_gpr[0] &= ~(1 << 17)

    return result_gpr

In [ ]:
def dtree_aggr(result_1, result_2, result_3, result_4, 
               result_5, result_6, result_7, result_8):
    
    path_class = [1, 4, 4, 1, 4, 1, 1, 4, 5, 1, 5, 1, 5, 5, 3, 3, 4, 1, 3, 1, 5, 5, 1, 1, 3, 4, 5, 3, 5, 1, 
                  5, 1, 5, 1, 4, 3, 4, 4, 5, 3, 3, 4, 5, 1, 4, 3, 5, 3, 4, 3, 4, 3, 5, 4, 3, 5, 4, 3, 3, 5, 
                  5, 5, 3, 4, 5, 5, 5, 3, 5, 3, 5, 5, 4, 3, 5, 5, 5, 3, 3, 2, 5, 3, 3, 4, 3, 3, 5, 5, 4, 4, 
                  2, 4, 4, 4, 2, 4, 2, 2, 2, 2, 3, 3, 4, 4, 2, 5, 2, 4, 3, 2, 2, 2, 3, 2]
    
    result_cpy = [0, 0, 0, 0]
    result_cpy[0] = result_1[0] & result_2[0] & result_3[0] & result_4[0] & result_5[0] & result_6[0] & result_7[0] & result_8[0]
    result_cpy[1] = result_1[1] & result_2[1] & result_3[1] & result_4[1] & result_5[1] & result_6[1] & result_7[1] & result_8[1]
    result_cpy[2] = result_1[2] & result_2[2] & result_3[2] & result_4[2] & result_5[2] & result_6[2] & result_7[2] & result_8[2]
    result_cpy[3] = result_1[3] & result_2[3] & result_3[3] & result_4[3] & result_5[3] & result_6[3] & result_7[3] & result_8[3]
    
    add_on = 0
    if (result_cpy[3] != 0):
        add_on = 0
        result_cpy[0] = result_cpy[3]
    elif (result_cpy[2] != 0):
        add_on = 32
        result_cpy[0] = result_cpy[2]
    elif (result_cpy[1] != 0):
        add_on = 64
        result_cpy[0] = result_cpy[1]
    else:
        add_on = 96
        result_cpy[0] = result_cpy[0]
    
    
    index = 0
    result_cpy[0] >>= 1
    while(result_cpy[0] != 0):
        index += 1
        result_cpy[0] >>= 1
    
    return path_class[index+add_on]

In [ ]:
clf_preds = clf.predict(X_test)
net_pred = []

In [ ]:
for i in range(len(X_test)):
    if_ex = [num_translate2(feat, total_bits, dec_bits) for feat in X_test[i]]
    
    result_1 = feature1(if_ex[0])
    result_2 = feature2(if_ex[1])
    result_3 = feature3(if_ex[2])
    result_4 = feature4(if_ex[3])
    result_5 = feature5(if_ex[4])
    result_6 = feature6(if_ex[5])
    result_7 = feature7(if_ex[6])
    result_8 = feature8(if_ex[7])
    if_pred = dtree_aggr(result_1, result_2, result_3, result_4, result_5, result_6, result_7, result_8)
    
    net_pred.append(if_pred)

In [ ]:
print(accuracy_score(y_test, net_pred))
print(f1_score(y_test, net_pred, average='macro'))
print(precision_score(y_test, net_pred, average='macro'))
print(recall_score(y_test, net_pred, average='macro'))